# Lasso, Ridge and Elastic Net regression

### Datset infromation

#### Algerian Forest Fires Dataset

The dataset includes 244 instances that regroup a data of two regions of ALgeria, namely the Bejaia region located in the norteast of Algeria and the Sidi Bel-abbes region located in the northwest of Algeria. 

122 instances for each region.

The period from June 2012 to Septermber 2012. The datset includes attributes and 1 output attribute (class). The 244 instances have been classified into fire (138 classes) and not fire (106 classes) classes.

**Link:** https://archive.ics.uci.edu/dataset/547/algerian+forest+fires+dataset

#### Variable Information
1. Date : (DD/MM/YYYY) Day, month ('june' to 'september'), year (2012)
Weather data observations 
2. Temp : temperature noon (temperature max)  in Celsius degrees: 22 to 42
3. RH : Relative Humidity in %: 21 to 90 
4. Ws :Wind speed in km/h: 6 to 29 
5. Rain: total day in mm: 0 to 16.8

**FWI Components**

6. Fine Fuel Moisture Code (FFMC) index from the FWI system: 28.6 to 92.5 
7. Duff Moisture Code (DMC) index from the FWI system: 1.1 to 65.9 
8. Drought Code (DC) index from the FWI system:  7 to 220.4
9. Initial Spread Index (ISI) index from the FWI system: 0 to 18.5 
10. Buildup Index (BUI) index from the FWI system: 1.1 to 68
11. Fire Weather Index (FWI) Index: 0 to 31.1
12. Classes: two classes, namely   fire and not fire

#### Data creator

Faroudja Abid

fabid@cdta.dz

Center for Development of Advanced Technologies (CDTA)

So main task it to know what Fire weather index (FWI) and Classes whether fire is goint to happen or not.

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import seaborn as sns 
%matplotlib inline 

In [2]:
df = pd.read_csv('Algerian_forest_fires_dataset_UPDATE.csv', header=1)

it will load first region data

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246 entries, 0 to 245
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   day          246 non-null    object
 1   month        245 non-null    object
 2   year         245 non-null    object
 3   Temperature  245 non-null    object
 4    RH          245 non-null    object
 5    Ws          245 non-null    object
 6   Rain         245 non-null    object
 7   FFMC         245 non-null    object
 8   DMC          245 non-null    object
 9   DC           245 non-null    object
 10  ISI          245 non-null    object
 11  BUI          245 non-null    object
 12  FWI          245 non-null    object
 13  Classes      244 non-null    object
dtypes: object(14)
memory usage: 27.0+ KB


so all the data is object type. SO we need to do some data engineering. For example converting the datatype, finding the missing values and infinties etc. 

In [4]:
df.isnull().sum()

day            0
month          1
year           1
Temperature    1
 RH            1
 Ws            1
Rain           1
FFMC           1
DMC            1
DC             1
ISI            1
BUI            1
FWI            1
Classes        2
dtype: int64

In [5]:
# Now let's see the location where data is missing:

df[df.isnull().any(axis=1)]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes
122,Sidi-Bel Abbes Region Dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
167,14,07,2012,37,37,18,0.2,88.9,12.9,14.6 9,12.5,10.4,fire,NaN


If we notice, it shows the data is missing on row 122, which is basically represening the data of different region. Since our data for the two regions are given in the datafile, we need to convert them two two different datsets. 

Now the dataset is converted into two sets based on region from 122th index, we can make a new column based on the region.
1. "Bejaia Region Datset"
2. "Sidi-Bel Abbes Region dataset"

add new column with region

Here we will lebel to first region to 0 and second region to 1.

In [11]:
# Identify the rows where the header for each region is located
bejaia_header_row = 1
sidi_bel_abbes_header_row = 126

In [12]:
# Extract data for the Bejaia region
bejaia_data = df.iloc[bejaia_header_row + 1:sidi_bel_abbes_header_row].copy()

# Extract data for the Sidi-Bel Abbes region
sidi_bel_abbes_data = df.iloc[sidi_bel_abbes_header_row + 1:].copy()

# Add a new column 'Region' with values 0 for the Bejaia region and 1 for the Sidi-Bel Abbes region
bejaia_data['Region'] = 0
sidi_bel_abbes_data['Region'] = 1

In [13]:
# Concatenate the two DataFrames back together
final_df = pd.concat([bejaia_data, sidi_bel_abbes_data])

# Reset the index of the final DataFrame
final_df.reset_index(drop=True, inplace=True)

In [14]:
final_df.head()

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
0,03,06,2012,26,82,22,13.1,47.1,2.5,7.1,0.3,2.7,0.1,not fire,0
1,04,06,2012,25,89,13,2.5,28.6,1.3,6.9,0,1.7,0,not fire,0
2,05,06,2012,27,77,16,0,64.8,3,14.2,1.2,3.9,0.5,not fire,0
3,06,06,2012,31,67,14,0,82.6,5.8,22.2,3.1,7,2.5,fire,0
4,07,06,2012,33,54,13,0,88.2,9.9,30.5,6.4,10.9,7.2,fire,0


In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   day          243 non-null    object
 1   month        242 non-null    object
 2   year         242 non-null    object
 3   Temperature  242 non-null    object
 4    RH          242 non-null    object
 5    Ws          242 non-null    object
 6   Rain         242 non-null    object
 7   FFMC         242 non-null    object
 8   DMC          242 non-null    object
 9   DC           242 non-null    object
 10  ISI          242 non-null    object
 11  BUI          242 non-null    object
 12  FWI          242 non-null    object
 13  Classes      241 non-null    object
 14  Region       243 non-null    int64 
dtypes: int64(1), object(14)
memory usage: 28.6+ KB


so we see that the Region is float type. But I dont want to save much memory for the float values for the region attribute. We can do it by converting the data type to intiger.

In [16]:
final_df.isna().sum()

day            0
month          1
year           1
Temperature    1
 RH            1
 Ws            1
Rain           1
FFMC           1
DMC            1
DC             1
ISI            1
BUI            1
FWI            1
Classes        2
Region         0
dtype: int64

In [17]:
final_df = final_df.dropna().reset_index(drop=True)

In [18]:
final_df.isna().sum()

day            0
month          0
year           0
Temperature    0
 RH            0
 Ws            0
Rain           0
FFMC           0
DMC            0
DC             0
ISI            0
BUI            0
FWI            0
Classes        0
Region         0
dtype: int64

In [20]:
final_df.iloc[122:130]

,day,month,year,Temperature,RH,Ws,Rain,FFMC,DMC,DC,ISI,BUI,FWI,Classes,Region
122,02,06,2012,30,73,13,4,55.7,2.7,7.8,0.6,2.9,0.2,not fire,0
123,04,06,2012,30,64,14,0,79.4,5.2,15.4,2.2,5.6,1,not fire,1
124,05,06,2012,32,60,14,0.2,77.1,6,17.6,1.8,6.5,0.9,not fire,1
125,06,06,2012,35,54,11,0.1,83.7,8.4,26.3,3.1,9.3,3.1,fire,1
126,07,06,2012,35,44,17,0.2,85.6,9.9,28.9,5.4,10.7,6,fire,1
127,08,06,2012,28,51,17,1.3,71.4,7.7,7.4,1.5,7.3,0.8,not fire,1
128,09,06,2012,27,59,18,0.1,78.1,8.5,14.7,2.4,8.3,1.9,not fire,1
129,10,06,2012,30,41,15,0,89.4,13.3,22.5,8.4,13.1,10,fire,1
